# Descripción del proyecto

La compañía Sweet Lift Taxi ha recopilado datos históricos sobre pedidos de taxis en los aeropuertos. Para atraer a más conductores durante las horas pico, necesitamos predecir la cantidad de pedidos de taxis para la próxima hora. Construye un modelo para dicha predicción.

La métrica RECM en el conjunto de prueba no debe ser superior a 48.

## Instrucciones del proyecto.

1. Descarga los datos y haz el remuestreo por una hora.
2. Analiza los datos
3. Entrena diferentes modelos con diferentes hiperparámetros. La muestra de prueba debe ser el 10% del conjunto de datos inicial.4. Prueba los datos usando la muestra de prueba y proporciona una conclusión.

## Descripción de los datos

Los datos se almacenan en el archivo `taxi.csv`. 	
El número de pedidos está en la columna `num_orders`.

## Preparación

#### Definir el tipo de proyecto

De acuerdo a las caracteristicas mencionadas del proycto, nos encontramos ante un objetivo de predicción a través de modelos de regresión.

#### Importación de librerias

In [57]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error
from pmdarima.arima import auto_arima
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
import math
import numpy as np

#### Importación de dataframe

In [2]:
# df = pd.read_csv('/datasets/taxi.csv',parse_dates=   ,index_col=)
df = pd.read_csv('D:/Tripleten/datasets/taxi.csv', index_col='datetime', parse_dates=[0] ) #
df.sort_index(inplace=True)

#### Visión general del df

In [3]:
print(df.head(), end='\n\n')
df.info()

                     num_orders
datetime                       
2018-03-01 00:00:00           9
2018-03-01 00:10:00          14
2018-03-01 00:20:00          28
2018-03-01 00:30:00          20
2018-03-01 00:40:00          32

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 26496 entries, 2018-03-01 00:00:00 to 2018-08-31 23:50:00
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   num_orders  26496 non-null  int64
dtypes: int64(1)
memory usage: 414.0 KB


#### Revision de duplicados

In [4]:
df.index.duplicated().sum()

0

#### Revisión de valores nulos

De acuerdo con df.info() no existen valores nulos

#### Coherencia de datos

Los datos mantienen una tendencia de crecimiento, por el momento muestran coherencia.

In [5]:
df.resample('W').sum().head(10)

,num_orders
datetime,
2018-03-04,5301
2018-03-11,9411
2018-03-18,9785
2018-03-25,9767
2018-04-01,9752
2018-04-08,9377
2018-04-15,10352
2018-04-22,11224
2018-04-29,12041


- [X] Identificar tipo de proyecto
- [X] Nombre de columna correcta
- [X] Clasificaciones de tipo de dato correctas
- [X] Dataframe sin duplicados
- [X] Excluir valores nulos
- [X] Verificar la coherencia de los datos

Conclusiones del proceso de preparación.

El conjunto de datos cargado mantiene una limpieza en sus datos (nombre de columna correctamente nombrados, coherencia de datos, inexistencia de duplicados y valores nulos ). Los datos cargados establecen un periodo desde el periodo 2018-03-01 hasta 2018-08-31.

## Análisis

In [6]:
# df.resample('YE').sum()
df_monthly = df.resample('ME').sum()
df_weekly = df.resample('W').sum()
df_daily = df.resample('D').sum()
df_hourly = df.resample('h').sum()

In [7]:
# titles = ['Gráfico mensual', 'Gráfico semanal', 'Gráfico diario', 'Gráfico por hora']
# interval = [1,4,24,480]

# databases = [df_monthly,df_weekly,df_daily, df_hourly]
# fig, axis = plt.subplots(4,1, figsize=[10,5])

# for i, database in enumerate(databases):
#     axis[i].plot(database.index, database.values, '--o')
#     axis[i].set_title(titles[i])
#     axis[i].set_xticks(database.index)
#     axis[i].set_xticks(database.index[::interval[i]])

# plt.tight_layout()
# plt.show()

In [8]:
decomposed = seasonal_decompose(df_hourly, model='additive', period =7)
# decomposed.plot()
# plt.title('Orders Trend')
# plt.show()

## Formación

Definiendo función para crear mas columnas de información

In [9]:
def make_features(df, max_lag, rolling_mean_size):
    '''Esta función tomará una serie temporal y creara columnas para aumentar su posibilidad de predicción
     en modelos de machine learning, generará columnas de año, mes, día, día de la semana y generará desfaces en los valores definidos 
    por la entrada, también generará una columna con el promedio móvil'''

    df['year'] = df.index.year
    df['month'] = df.index.month
    df['day'] = df.index.day
    df['dayofweek'] = df.index.dayofweek

    for lag in range(1, max_lag +1):
        df[f'lag_{lag}'] = df['num_orders'].shift(lag)

    # df['rolling_mean'] = df['num_orders'].rolling(rolling_mean_size).mean()
    df['rolling_mean'] = df['num_orders'].shift().rolling(rolling_mean_size).mean()

make_features(df_hourly,3,3)

df_hourly.dropna(inplace=True)

In [10]:

y= df_hourly['num_orders']
X= df_hourly.drop(columns='num_orders')

X_train, X_test,y_train, y_test = train_test_split(X,y,shuffle=False, test_size=1/10) #10% del conjuntos de datos inicial

## Prueba

#### Modelo Linear Regression

In [76]:

def linear(X_train, y_train, X_test,y_test):
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Prediciendo las muestras
    y_predict = model.predict(X_test)
    test_rmse_score = root_mean_squared_error(y_test,y_predict)

    objective_rmse_score = root_mean_squared_error([y_test.iloc[0]],[y_predict[0]])

    print(f'RMSE para muestra es:{test_rmse_score:.2f}')
    print(f'RMSE para la siguiente hora:{objective_rmse_score:.2f}')
    print(f'Cantidad de taxis predichos para la siguiente hora es de "{round(y_predict[0])}" con un error de "{round(objective_rmse_score)}"')

In [77]:
linear(X_train, y_train, X_test,y_test)

RMSE para muestra es:52.45
RMSE para la siguiente hora:6.02
Cantidad de taxis predichos para la siguiente hora es de "108" con un error de "6"


#### Modelo Auto Arima

In [79]:
df_arima = df_hourly['num_orders']
train, test = train_test_split(df_arima, shuffle=False, test_size=1/10)

In [118]:
model = auto_arima(train, seasonal=True, m=12, stepwise=True)
summary = model.summary()
y_predict = model.predict(n_periods=5) #len(test) 442

rmse_score = root_mean_squared_error(test.iloc[0:5:], y_predict)
objective_rmse_score = root_mean_squared_error([test.iloc[0]], [y_predict.iloc[0]])

print(f'RMSE para muestra es:{rmse_score:.2f}')
print(f'RMSE para la siguiente hora:{objective_rmse_score:.2f}')
print(f'Cantidad de taxis predichos para la siguiente hora es de "{y_predict.iloc[0]:.2f}" con un error de "{round(objective_rmse_score)}"')


RMSE para muestra es:33.60
RMSE para la siguiente hora:32.45
Cantidad de taxis predichos para la siguiente hora es de "134.45" con un error de "32"


#### Modelo Prophet (Meta)

In [125]:
df_prophet = pd.DataFrame(data={"ds":df_hourly.index,"y":df_hourly['num_orders']})
df_prophet = df_prophet.reset_index(drop=True)

train, test =train_test_split(df_prophet, test_size=1/10, shuffle=False)

model = Prophet()
model.fit(train)

# Realizar las predicciones para el conjunto de prueba
future = model.make_future_dataframe(periods=len(test), freq='h')
forecast = model.predict(future)

y_predict = forecast['yhat'].tail(len(test))
rmse_score = root_mean_squared_error(test['y'],y_predict)
objective_rmse_score = root_mean_squared_error([test['y'].iloc[0]],[y_predict.iloc[0]])

print(f'RMSE para muestra es:{rmse_score:.2f}')
print(f'RMSE para la siguiente hora:{objective_rmse_score:.2f}')
print(f'Cantidad de taxis predichos para la siguiente hora es de "{y_predict.iloc[0]}" con un error de "{round(objective_rmse_score)}"')

12:51:09 - cmdstanpy - INFO - Chain [1] start processing
12:51:09 - cmdstanpy - INFO - Chain [1] done processing


RMSE para muestra es:48.27
RMSE para la siguiente hora:13.06
Cantidad de taxis predichos para la siguiente hora es de "115.05502204867386" con un error de "13"


#### Modelo Random Forest Regressor

# Lista de revisión

- [x]  Jupyter Notebook está abierto.
- [ ]  El código no tiene errores
- [ ]  Las celdas con el código han sido colocadas en el orden de ejecución.
- [ ]  Los datos han sido descargados y preparados.
- [ ]  Se ha realizado el paso 2: los datos han sido analizados
- [ ]  Se entrenó el modelo y se seleccionaron los hiperparámetros
- [ ]  Se han evaluado los modelos. Se expuso una conclusión
- [ ]  La *RECM* para el conjunto de prueba no es más de 48